In [1]:
import numpy as np
from typing import Tuple, Generator, Iterator

In [8]:
class ColumnBunches3():
    
    def __init__(self, data: np.ndarray, window: int, no_diag=False, diag_only=False):
        
        self.data = data
        self.N = data.shape[1]
        self.w = window
        self.nd = int(no_diag)
        
        # I, J indeces identify bunches of w columns
        # they generally sit in range(0:M)
        self.M = (self.N-1) // self.w + 1
        self.IJK = self._IJK_gen() if not diag_only else self._IJK_diag_gen()
        
    def _IJK_gen(self) -> Generator:
        for I in range(self.M):
            for J in range(I + self.nd, self.M):
                for K in range(I + J + self.nd, self.M):
                    yield I, J, K
                
    def _IJK_diag_gen(self) -> Generator:
        for I in range(self.M):
            for J in range(I + self.nd, self.M):
                yield I, J, (self.M-1) - (self.M-1 - I) - (self.M-1 - J)
    
    def _get_columns(self, I: int, J: int, K: int) -> Tuple:
        return self.data[:, self.w*I : self.w*(I+1)], \
               self.data[:, self.w*J : self.w*(J+1)], \
               self.data[:, self.w*K : self.w*(K+1)]
    
    def __next__(self) -> Tuple:
        return self._get_columns(*next(self.IJK))
    
    def __iter__(self) -> Iterator:
        return self

In [9]:
M1 = np.array([[1.,2,3,4,5]]*3)
M1

array([[1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.]])

In [10]:
bunches = ColumnBunches3(M1, window=3)

for bunch in bunches:
    for column in bunch:
        print(column)
    print(">-------")

[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
>-------
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------


In [12]:
bunches = ColumnBunches3(M1, window=3, no_diag=True) # None

for bunch in bunches:
    for column in bunch:
        print(column)
    print(">-------")

In [13]:
bunches = ColumnBunches3(M1, window=3, diag_only=True)

for bunch in bunches:
    for column in bunch:
        print(column)
    print(">-------")

[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[]
>-------
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]
>-------
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
[[4. 5.]
 [4. 5.]
 [4. 5.]]
>-------


In [31]:
[2,3,1].sort()